## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD could come from any datasource, e.g. text files, a database, a JSON file etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependancy graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high level spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="rdd1.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation a RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check : confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1000))
len(data)

#1000

1000

### Initialize an RDD

To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

0

In [8]:
# take
rdd.take(5)

[0, 1, 2, 3, 4]

In [9]:
# top
rdd.top(5)

[999, 998, 997, 996, 995]

In [ ]:
# collect
rdd.collect()

## Map functions

Now that you've working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new ecommerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 by a random number from 0-1.

In [11]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([5.17651509e-01, 8.73714039e-01, 1.65671711e+00, 3.24051882e+00,
       4.00946323e+00, 9.28771928e-01, 5.82539650e+00, 2.84444218e+00,
       1.04623970e+00, 7.52063287e+00, 8.61113042e+00, 3.43873580e+00,
       1.01663006e+01, 1.15659614e+01, 9.67182748e+00, 7.70419758e+00,
       1.21905222e-01, 1.69404319e+00, 2.42863124e+00, 6.00355318e+00,
       1.71650852e+01, 1.97208607e+01, 1.29202594e+01, 6.92067361e-01,
       1.82862426e+01, 1.42308923e+01, 2.59284334e+01, 7.07134609e-01,
       1.70923238e+01, 1.30925540e+01, 3.03916879e+01, 1.38638573e+01,
       4.93306693e-01, 1.74152838e+00, 7.99505230e+00, 9.28989365e+00,
       1.13613675e+01, 3.44448831e+01, 7.26000526e+00, 1.31643442e+00,
       2.45091933e+01, 4.09788668e+01, 3.08555191e+01, 3.25438270e+01,
       3.61836676e+01, 5.82344490e+00, 4.22739132e+00, 3.93520152e+01,
       4.30917424e+01, 1.96473654e+01, 2.66768409e+01, 5.63712489e+00,
       4.47403489e+01, 1.02889477e+01, 9.85448574e+00, 4.60429395e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [ ]:
price_items = sc.parallelize(sales_figures, 10)

In [14]:
type(price_items)

pyspark.rdd.RDD

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [18]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call use an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [21]:
revenue_minus_tax.take(10)

[0.4762393879936447,
 0.8038169161905011,
 1.5241797419335363,
 2.981277316820199,
 3.688706168965071,
 0.8544701740685743,
 5.35936477833035,
 2.616886806160029,
 0.962540524627943,
 6.9189822436210076]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [24]:
discounted = price_items.map(lambda x: (x * .9))

In [25]:
discounted.take(10)

[0.4658863578198698,
 0.786342635403751,
 1.4910453997175899,
 2.916466940367586,
 3.608516904422352,
 0.8358947355018661,
 5.242856848366646,
 2.5599979625478544,
 0.9416157306142922,
 6.768569586150985]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [ ]:
discount_and_tax = price_items.map(lambda x: sales_tax(x * .9))

In [27]:
discount_and_tax.top(15)

[797.344959618921,
 789.0090941265034,
 782.7650464904589,
 779.2907195316734,
 773.9378610792585,
 766.4805995958558,
 759.6864847402052,
 748.9739367509171,
 746.8298099479781,
 726.3291050473127,
 707.6909153317522,
 702.4037075799699,
 701.3710212623578,
 697.8649454157742,
 696.3041642744115]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [28]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [29]:
mapped = price_items.map(lambda x: (x, x * .9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.5176515086887442, 0.4658863578198698), (0.8737140393375011, 0.786342635403751), (1.656717110797322, 1.4910453997175899), (3.240518822630651, 2.916466940367586), (4.009463227135947, 3.608516904422352), (0.9287719283354068, 0.8358947355018661), (5.825396498185162, 5.242856848366646), (2.844442180608727, 2.5599979625478544), (1.0462397006825468, 0.9416157306142922), (7.5206328735010946, 6.768569586150985)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [30]:
flat_mapped = price_items.flatMap(lambda x: (x, x * .9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.5176515086887442, 0.4658863578198698, 0.8737140393375011, 0.786342635403751, 1.656717110797322, 1.4910453997175899, 3.240518822630651, 2.916466940367586, 4.009463227135947, 3.608516904422352]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criteria. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [31]:
# use the filter function
selected_items = discount_and_tax.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

262

## Reduce

Now it's time to figure out how much money BuyStuff would make from selling one of all of it's items after they've reduced their inventory. Use a reduce method with a lambda function to to add up all of the values in the RDD. Your lambda function should have two variables.

In [34]:
selected_items.reduce(lambda x, y: x + y)

120668.38504182789

The time has come for BuyStuff to open up shop and start selling it's goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [35]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(37, 312.41425541203944),
 (37, 327.6601368809986),
 (17, 334.4697841063853),
 (30, 344.77199584321903),
 (41, 347.7340275286324),
 (18, 306.07970814225666),
 (30, 352.2889200329263)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from highest spending to least spending.



In [39]:
# calculate how much each user spent
sales_data.reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1], ascending=False).collect()

[(10, 7984.4965109665045),
 (13, 7271.665636593777),
 (38, 6932.02067753431),
 (29, 6026.989516661704),
 (20, 5919.110182309427),
 (37, 5890.735551530993),
 (33, 5717.668809150444),
 (30, 5629.874509036534),
 (19, 5376.703056663366),
 (50, 4901.872175585835),
 (31, 4733.945047030082),
 (11, 4543.468108880126),
 (28, 4535.29908253022),
 (39, 4510.664692751573),
 (18, 4219.913751707682),
 (26, 3379.696882660315),
 (49, 3189.544376809883),
 (25, 2984.234813105253),
 (3, 2816.886614209128),
 (45, 2805.3053905311244),
 (23, 2675.647323200131),
 (40, 2470.8376982020154),
 (17, 2406.4510155876033),
 (41, 2398.189289524894),
 (7, 2290.7148426917533),
 (21, 2099.411831069594),
 (46, 1788.6279511665275),
 (32, 1346.1509561276594),
 (44, 1166.009342977397),
 (8, 1013.7765554448616),
 (6, 638.262221266879),
 (15, 526.7564777255802),
 (2, 477.4541505947121)]

In [ ]:
# sort the users from highest to lowest spenders


Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [43]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(8, 15),
 (31, 13),
 (13, 10),
 (43, 10),
 (2, 10),
 (17, 9),
 (15, 9),
 (41, 8),
 (39, 8),
 (7, 8),
 (32, 8),
 (34, 8),
 (40, 8),
 (28, 7),
 (29, 7),
 (9, 7),
 (36, 6),
 (46, 6),
 (24, 6),
 (48, 6),
 (26, 5),
 (11, 5),
 (49, 5),
 (38, 5),
 (33, 5),
 (45, 5),
 (19, 4),
 (44, 4),
 (22, 4),
 (14, 4),
 (20, 4),
 (4, 4),
 (10, 4),
 (25, 3),
 (27, 3),
 (42, 3),
 (21, 3),
 (1, 3),
 (30, 3),
 (6, 3),
 (50, 3),
 (35, 3),
 (3, 2),
 (47, 2),
 (16, 1),
 (18, 1),
 (5, 1),
 (23, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
